In [1]:
import pandas as pd
import os

In [2]:
path = f'{os.getcwd()}/PDBs'

In [17]:
df = pd.read_csv(f'{path}/APD3/APD3_2023_02_02.csv')

In [57]:
def multi_replace(text, replaces) -> (str, dict):
    for key in replaces.keys():
        text = text.replace(key, replaces[key])
    return text

In [61]:
# Obtem os DOI que ja estao no DataFrame

doi = {
    'Index': [],
    'DOI': []
}

#df['Reference'][2745] = 'Not Reference'
replaces = {
    'doi: ': 'doi:',
    '.Pub': ' ',
    'doi:': 'doi: ',
    'doi.org/': 'doi: ',
    '.Abs': ' '
}

for i in range(0, len(df['Reference'])):
    for ref_info in df['Reference'][i].split('. '):
        if 'doi' in ref_info:
            ref_info = multi_replace(ref_info, replaces)
            doi['Index'].append(i)
            doi['DOI'].append(ref_info.split('doi:')[1].split(' ')[1])
            #print(i, '-', ref_info.split('doi:')[1].split(' ')[1])

741

In [69]:
references = {
    'Name': [],
    'Count': []
}

replaces = {
    'Pub-Med': 'PubMed',
    ' PubMed': 'PubMed',
    ' Pub-Med': 'PubMed'
}

for ref in df['Reference']:
    # Trata um erro no index 2745 (value=nan)
    if type(ref) == str:
        refs = ref.split('.')
    else:
        continue
        
    # Obtem o nome do site de referencia
    ref_info = [x for x in refs if x != '']
    ref_name = ref_info[len(ref_info)-1]
    
    # Altera o nome 'PubMed', que aparece de formas diferentes
    if ref_name in replaces.keys():   
        ref_name = multi_replace(ref_name, replaces)
    
    # Verifica se o nome ja esta na lista e altera a contagem
    exists = [x for x in references['Name'] if ref_name == x]
    if not exists:
        references['Name'].append(ref_name)
        references['Count'].append(1)
    else:
        for i in range(0, len(references['Name'])-1):
            if references['Name'][i] == ref_name:
                references['Count'][i] += 1

In [70]:
# Count of reference sites
for i in range(0, len(references['Count'])):
    if references['Count'][i] > 2:
        print(references['Count'][i])
        print(references['Name'][i])

2622
PubMed
4
 2008 Dec;76(12):5883-91
3
Submitted JUL-2002 to the SWISS-PROT data bank
3
 2010 Jun;31(6):989-94
3
 1988 Jun 5;263(16):7472-7
3
Peptides 2006; 27: 1738-1744
3
 2004 Jul 6;43(26):8391-409
3
 2008 Nov;29(11):2074-82
4
 2003 Nov 15;116(1-3):139-46
3
 1999;20(6):679-86
3
 1999 Sep 10;274(37):26172-8
25
1
3
 2010 May;92(5):475-81
7
 ScienceDirect
3
 2006 May;47(6):664-75
6
2
11
 Sci Direct
6
 PDF
5
 Publisher
44
 MDPI
